<a href="https://colab.research.google.com/github/adlihs/instatXML/blob/main/Instat_XML_file_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process to work with Instat's xml file

## Import libraries

In [ ]:
!pip install pandas_read_xml
!pip install -U pandasql
import pandas_read_xml as pdx 
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import os,glob 
import pandas as pd
import time
import numpy as np
from pandasql import sqldf 

## Setup the name of the tournament

In [16]:
nombre_torneo = "Clausura 2022"

## Setup the google drive folder's path
The idea is to download all the xml from Instat and put all in a google drive folder.

In [7]:
%cd "/content/drive/My Drive/XML Clausura 2022" #this is an example of my folder's path

/content/drive/.shortcut-targets-by-id/1DUFFBYsGBYbZRixKmbuaHmn23MJMc0yg/XML Clausura 2022


## Remove excel files from previous process
The script will generate an excel file for each xml, so the idea is to remove the files from previous process.

In [8]:
print('Finding and removing old excel files...')
excel_files = []
for file in glob.glob("*.xlsx"):
    excel_files.append(file)

for file in excel_files:
  os.remove(file)

print('done!')

Finding and removing old excel files...
done!


## Let's find all the XML files in the folder

In [9]:
print('Finding xml files...')
xml_files = []
for file in glob.glob("*.xml"):
    xml_files.append(file)

print('done!')

Finding xml files...
done!


## Generate an excel file for each xml file

In [10]:
print('Creating excel files from xmls')
open_file =""
for xml in xml_files:
  # This line will remove the name of the competion from the xml file name
  file_name = xml.replace('--Costa-Rica--Primera-Division','')
  # This line will remove the text '.xml' from the variable 'file_name' 
  file_name = file_name.replace('.xml','')
  # This line will create a dataframe with the xml data
  open_file = pdx.read_xml(xml, ['file', 'ALL_INSTANCES', 'instance'], root_is_rows=False)
  # This line is to create a new column named 'file_name', the column will contain the information of the game
  open_file["file_name"] = file_name
  # This line will generate an excel file
  open_file.to_excel(file_name + ".xlsx")
  #print(open_file.file_name)

print('done!')
open_file

Creating excel files from xmls
done!


,ID,start,end,code,label,pos_x,pos_y,file_name
0,1,5.15,25.15,7. Allen Esteban Guevara Zuniga,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",52.5,34.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
1,2,5.15,25.15,7. Allen Esteban Guevara Zuniga,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",52.5,34.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
2,3,5.63,25.63,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",52.5,33.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
3,4,6.74,26.74,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",52.8,33.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
4,5,6.74,26.74,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",52.8,33.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
...,...,...,...,...,...,...,...,...
2907,2908,5565.21,5585.21,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",66.2,45.5,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
2908,2909,5565.21,5585.21,15. Celso Borges,"[{'group': 'Equipo', 'text': 'Alajuelense'}, {...",38.8,22.5,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
2909,2910,5565.21,5585.21,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",66.2,45.5,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
2910,2911,5593.86,5613.86,26. Diego Andres Sanchez Corrales,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",14.6,0.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...


## Merge all excel files in one dataframe

In [11]:
print('Merging files...')
all_data = pd.DataFrame()
for f in glob.glob("*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)
print('done!')
all_data

Merging files...
done!


,Unnamed: 0,ID,start,end,code,label,pos_x,pos_y,file_name
0,0,1,7.73,27.73,88. Starling Matarrita Gonzalez,"[{'group': 'Equipo', 'text': 'Perez Zeledon'},...",52.2,35.2,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...
1,1,2,7.73,27.73,88. Starling Matarrita Gonzalez,"[{'group': 'Equipo', 'text': 'Perez Zeledon'},...",52.2,35.2,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...
2,2,3,8.52,28.52,20. Nestor Monge Guevara,"[{'group': 'Equipo', 'text': 'Perez Zeledon'},...",37.9,34.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...
3,3,4,8.52,28.52,20. Nestor Monge Guevara,"[{'group': 'Equipo', 'text': 'Perez Zeledon'},...",37.9,34.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...
4,4,5,10.52,30.52,23. Mauricio de Jesus Nunez Morales,"[{'group': 'Equipo', 'text': 'Perez Zeledon'},...",25.4,54.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...
...,...,...,...,...,...,...,...,...,...
502108,2907,2908,5565.21,5585.21,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",66.2,45.5,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
502109,2908,2909,5565.21,5585.21,15. Celso Borges,"[{'group': 'Equipo', 'text': 'Alajuelense'}, {...",38.8,22.5,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
502110,2909,2910,5565.21,5585.21,9. Marcel Hernandez Campanioni,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",66.2,45.5,26-06-2022Cartagines-0-0-Alajuelense20220628-8...
502111,2910,2911,5593.86,5613.86,26. Diego Andres Sanchez Corrales,"[{'group': 'Equipo', 'text': 'Cartagines'}, {'...",14.6,0.0,26-06-2022Cartagines-0-0-Alajuelense20220628-8...


## Split column `'Label'` by comma `','`

In [12]:
#Create new dataframe with the new columns from the column 'LABEL'
print('Creating new data frame')
all_data2 = all_data['label'].str.split(',', expand=True)


#Merge original dataframe and new dataframe
print('Merging dataframes')
all_data = pd.concat([all_data, all_data2], axis=1, sort=False)

###REMOVE UNNCESSARY COLUMNS
print('Removing columns')
all_data.drop('label', inplace=True, axis=1)
all_data.drop(0, inplace=True, axis=1)
all_data.drop(2, inplace=True, axis=1)
all_data.drop(4, inplace=True, axis=1)

#### RENAME NEW COLUMNS (Teams, Actions, Time)
print('Reanaming columns')
all_data = all_data.rename(columns={1: "Equipos", 3: "Acciones",5:"Tiempo"})


#### REPLACE CHARACTERS IN COLUMNS EQUIPOS, ACCIONES, TIEMPO
print('Replacing special characters')
import re
chars_to_remove = ['.', '-', '(', ')',']','\'', '','}',':']
regular_expression = '[' + re.escape (''. join (chars_to_remove)) + ']'

all_data['Equipos'] = all_data['Equipos'].str.replace(regular_expression, '', regex=True)
all_data['Acciones'] = all_data['Acciones'].str.replace(regular_expression, '', regex=True)
all_data['Tiempo'] = all_data['Tiempo'].str.replace(regular_expression, '', regex=True)

all_data['Equipos'] = all_data['Equipos'].str.replace('text','')
all_data['Acciones'] = all_data['Acciones'].str.replace('text','')
all_data['Tiempo'] = all_data['Tiempo'].str.replace('text','')


print('done!')


Creating new data frame
Merging dataframes
Removing columns
Reanaming columns
Replacing special characters
done!


## Preview the dataframe

In [14]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,file_name,Equipos,Acciones,Tiempo
0,0,1,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Ataques posicionales,1ª parte
1,1,2,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Pases precisos,1ª parte
2,2,3,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Ataques posicionales,1ª parte
3,3,4,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Pases precisos,1ª parte
4,4,5,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Ataques posicionales,1ª parte
5,5,6,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Pases precisos,1ª parte
6,6,7,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Ataques posicionales,1ª parte
7,7,8,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Pases precisos,1ª parte
8,8,9,16.66,36.66,23. Mauricio de Jesus Nunez Morales,22.5,59.4,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,Perez Zeledon,Ataques posicionales,1ª parte
9,9,10,17.73,18.73,Empezar marca de tiempo,NaN,NaN,13-01-2022Perez-Zeledon-1-1-Municipal-Grecia20...,NaN,NaN,NaN


## Get new columns from `file_name` column

In [17]:
#New column 'fecha_partido' (game_date)
print('creating new column fecha_partido')
all_data['fecha_partido'] = all_data['file_name'].str[:10]


#New column 'nombre_torneo' (tournament_name)
print('creating new column nombre_torneo')
all_data['nombre_torneo'] = nombre_torneo

#remove game date from string
print('Removing game date from string')
all_data['file_name'] = all_data['file_name'].str[10:]


#Remove last 20 characters
print('removing unnecessary last 22 last characters')
all_data['file_name'] = all_data['file_name'].str[:-21]

####RENAME COLUMN FILE_NAME
print('Reanaming column')
all_data = all_data.rename(columns={'file_name': "Partido"})

print('done!')

creating new column fecha_partido
creating new column nombre_torneo
Removing game date from string
removing unnecessary last 22 last characters
Reanaming column
done!


## Preview the dataframe with the new columns

In [19]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,fecha_partido,nombre_torneo
0,0,1,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022
1,1,2,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022
2,2,3,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022
3,3,4,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022
4,4,5,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022
5,5,6,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022
6,6,7,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022
7,7,8,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022
8,8,9,16.66,36.66,23. Mauricio de Jesus Nunez Morales,22.5,59.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022
9,9,10,17.73,18.73,Empezar marca de tiempo,NaN,NaN,Perez-Zeledon-1-1-Municipal-Grecia,NaN,NaN,NaN,13-01-2022,Clausura 2022


## Remove white spaces from the column `Acciones` (actions)

In [31]:
#TRIM spaces
all_data['Acciones'] = all_data['Acciones'].str.strip()
all_data['Equipos'] = all_data['Equipos'].str.strip()

## Setup columns with the `home team name` and the `away team name`

In [20]:
# Replace score in game description with @
all_data["partido_text"] = all_data["Partido"].replace(to_replace=r'[-]+\d[-]+\d+[-]', value='@', regex=True)

# Create a temp dataframe with columns 'home_team'/'away_team'
localia_df = all_data['partido_text'].str.split('@', expand=True)

# Merge new temp dataframe with all_data dataframe
all_data = pd.concat([all_data, localia_df], axis=1, sort=False)

#Rename new columns
all_data.rename(columns={0: 'equipo_casa', 1: 'equipo_visita'}, inplace=True)

#Drop column 'partido_text'
all_data.drop('partido_text', inplace=True, axis=1)

## Preview the dataframe with the new changes

In [21]:
all_data.head(20)

,Unnamed: 0,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,fecha_partido,nombre_torneo,equipo_casa,equipo_visita
0,0,1,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
1,1,2,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
2,2,3,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
3,3,4,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
4,4,5,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
5,5,6,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
6,6,7,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
7,7,8,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
8,8,9,16.66,36.66,23. Mauricio de Jesus Nunez Morales,22.5,59.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia
9,9,10,17.73,18.73,Empezar marca de tiempo,NaN,NaN,Perez-Zeledon-1-1-Municipal-Grecia,NaN,NaN,NaN,13-01-2022,Clausura 2022,Perez-Zeledon,Municipal-Grecia


## Get columns for: **`Score`**, **`Home and Away goals`**

In [22]:
# Get the score from column 'Partido' (game)
all_data["marcador"] = all_data['Partido'].str.findall(r'\d[-]+\d+')

#convert column 'marcador' to string
all_data["marcador"] = all_data["marcador"].astype('str') 

# marcador en formato string
all_data["marcador"] = all_data["marcador"].str.slice(start=2, stop=-2)


#New column with the home team goals ('goles_local')
all_data['goles_local'] = all_data["marcador"].str.slice(start=0, stop=1)

#New column with the away team goals ('goles_visita')
all_data['goles_visita'] = all_data["marcador"].str.slice(start=2, stop=3)




## Preview the dataframe with the changes

In [49]:
all_data.head(20)

,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,...,goles_visita,rival,marcador_casa,marcador_visita,detalle_marcador,start_min,marcador_casa,marcador_visita,detalle_marcador,start_min
0,1,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
1,2,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
2,3,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
3,4,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
4,5,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
5,6,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
6,7,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
7,8,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
8,9,16.66,36.66,23. Mauricio de Jesus Nunez Morales,22.5,59.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
9,10,17.73,18.73,Empezar marca de tiempo,NaN,NaN,Perez-Zeledon-1-1-Municipal-Grecia,None,None,None,...,1,0,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542


## Setup a column with the team that "suffers" the action


In [43]:
# create a list of our conditions

conditions = [
    (all_data["Equipos"] == all_data["equipo_casa"]),
    (all_data["Equipos"] == all_data["equipo_visita"])]

# create a list of the values we want to assign for each condition
values = [all_data["equipo_visita"], all_data["equipo_casa"]]

# create a new column and use np.select to assign values to it using our lists as arguments
all_data["rival"] = np.select(conditions, values)


#### --- Equipo visita standarization name --- #####
replace_values = {'Alajuelens' : 'Alajuelense', 
                  'Alajuelense2' : 'Alajuelense', 
                  'Cartagine' : 'Cartagines',
                  'Cartagines2' : 'Cartagines', 
                  'Deportivo-Saprissa' : 'Saprissa',
                  'Deportivo-Saprissa2' : 'Saprissa', 
                  'Deportivo-Sapriss' : 'Saprissa', 
                  'Guadalupe2' : 'Guadalupe',
                  'Guanacasteca2' : 'Guanacasteca', 
                  'Guanacastec' : 'Guanacasteca',
                  'Herediano2' : 'Herediano',
                  'Heredia' : 'Herediano',
                  'Jicaral2' : 'Jicaral', 
                  'Municipal-Greci' : 'Municipal Grecia',
                  'Municipal-Grecia' : 'Municipal Grecia', 
                  'Municipal-Grecia2' : 'Municipal Grecia',
                  'Perez-Zeled' : 'Perez Zeledon', 
                  'Perez-Zeledo' : 'Perez Zeledon',
                  'Perez-Zeledon' : 'Perez Zeledon', 
                  'Perez-Zeledon2' : 'Perez Zeledon',
                  'San-Carlos' : 'San Carlos', 
                  'San-Carlos2' : 'San Carlos',
                  'Santos-de-Guapiles' : 'Santos de Guapiles', 
                  'Santos-de-Guapiles2' : 'Santos de Guapiles',
                  'Santos-de-Guapil' : 'Santos de Guapiles',
                  'Santos-de-Guapile' : 'Santos de Guapiles',
                  'Sporting-F' : 'Sporting FC', 
                  'Sporting-FC-San-Jos' : 'Sporting FC', 
                  'Sporting-FC-San-Jose' : 'Sporting FC', 
                  'Sporting-FC-San-Jose2' : 'Sporting FC', 
                  'Sporting-FC' : 'Sporting FC',
                  'Sporting-FC2' : 'Sporting FC'} 

all_data = all_data.replace({"rival": replace_values})

## Preview the changes

In [48]:
all_data.head(30)

,ID,start,end,code,pos_x,pos_y,Partido,Equipos,Acciones,Tiempo,...,goles_visita,rival,marcador_casa,marcador_visita,detalle_marcador,start_min,marcador_casa,marcador_visita,detalle_marcador,start_min
0,1,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
1,2,7.73,27.73,88. Starling Matarrita Gonzalez,52.2,35.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
2,3,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
3,4,8.52,28.52,20. Nestor Monge Guevara,37.9,34.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
4,5,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
5,6,10.52,30.52,23. Mauricio de Jesus Nunez Morales,25.4,54.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
6,7,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
7,8,13.14,33.14,55. Chistian Antonio Reyes Aleman,17.7,44.2,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Pases precisos,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
8,9,16.66,36.66,23. Mauricio de Jesus Nunez Morales,22.5,59.4,Perez-Zeledon-1-1-Municipal-Grecia,Perez Zeledon,Ataques posicionales,1ª parte,...,1,Municipal Grecia,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542
9,10,17.73,18.73,Empezar marca de tiempo,NaN,NaN,Perez-Zeledon-1-1-Municipal-Grecia,None,None,None,...,1,0,0.0,0.0,0-0,NaN,0.0,1.0,0-1,7.542


## Remove unnecesary columns

In [47]:
all_data.drop(all_data.columns[0], axis=1,inplace=True)

## Export the consolidated dataframe to Excel file

In [ ]:

print('Creating excel file with the dataframe all_data')
all_data.to_excel('UNAFUT_'+nombre_torneo+'.xlsx') # YOU CAN SETUP THE FILE NAME THAT YOU WANT
print('done!')

## Delete all excel files except the file created in the previous step

In [ ]:
print('Detecting all .xlsx files')
excel_files = []
for file in glob.glob("*.xlsx"):
    excel_files.append(file)

print('Removing all excel files except UNAFUT_' + nombre_torneo +'.xlsx')
for file in excel_files:
  if file != ('UNAFUT_' + nombre_torneo + '.xlsx') : # HERE CHANGE AND PUT THE FILE NAME YOU PUT IN THE PREVIOUS STEP
    os.remove(file)

print('All process is done!')